In [109]:
import numpy as np
import pandas as pd
import seaborn as sns

import geopandas as gpd

import shapely.wkt

from matplotlib import pyplot as plt

<h1>Índice<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Justificativa" data-toc-modified-id="Justificativa-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Justificativa</a></span></li><li><span><a href="#Carrega-os-dados" data-toc-modified-id="Carrega-os-dados-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Carrega os dados</a></span></li><li><span><a href="#Identificar-escolas-que-saltaram-de-IDEBs-baixo/baixíssimo-(&lt;-5/&lt;-4)-e-foram-para-IDEB-alto-(&gt;-6)-no-período." data-toc-modified-id="Identificar-escolas-que-saltaram-de-IDEBs-baixo/baixíssimo-(<-5/<-4)-e-foram-para-IDEB-alto-(>-6)-no-período.-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Identificar escolas que saltaram de IDEBs baixo/baixíssimo (&lt; 5/&lt; 4) e foram para IDEB alto (&gt; 6) no período.</a></span></li><li><span><a href="#Ver-a-visualização-de-dados-olhando-apenas-para-as-escolas-com-IDEB-inicial-baixo-(&lt;-5)-ou-baixíssimo-(&lt;-4);" data-toc-modified-id="Ver-a-visualização-de-dados-olhando-apenas-para-as-escolas-com-IDEB-inicial-baixo-(<-5)-ou-baixíssimo-(<-4);-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Ver a visualização de dados olhando apenas para as escolas com IDEB inicial baixo (&lt; 5) ou baixíssimo (&lt; 4);</a></span></li><li><span><a href="#Separar-os-dados-em-4-extratos-de-IDEB-para-podermos-ter-uma-noção-de-qual-é-o-nível-de-atenção-que-precisa-ser-dispensado-a-cada-escola;" data-toc-modified-id="Separar-os-dados-em-4-extratos-de-IDEB-para-podermos-ter-uma-noção-de-qual-é-o-nível-de-atenção-que-precisa-ser-dispensado-a-cada-escola;-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Separar os dados em 4 extratos de IDEB para podermos ter uma noção de qual é o nível de atenção que precisa ser dispensado a cada escola;</a></span></li><li><span><a href="#Ver-escolas-que-evoluíram-bem-vizinhas-de-escolas-nos-maiores-estratos-de-risco" data-toc-modified-id="Ver-escolas-que-evoluíram-bem-vizinhas-de-escolas-nos-maiores-estratos-de-risco-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Ver escolas que evoluíram bem vizinhas de escolas nos maiores estratos de risco</a></span></li></ul></div>

# Justificativa

Nosso objetivo final é propor um plano de recuperação para as escolas do município do Rio que estão com rendimento ruim no IDEB. Pra isso, o Ragazzo sugeriu algumas ideias:

1) Identificar escolas que saltaram de IDEBs baixo/baixíssimo (< 5/< 4) e foram para IDEB alto (> 6) no período.

2) Ver a visualização de dados olhando apenas para as escolas com IDEB inicial baixo (< 5) ou baixíssimo (< 4);

3) Separar os dados em 4 extratos de IDEB para podermos ter uma noção de qual é o nível de atenção que precisa ser dispensado a cada escola;

4) Ver escolas que evoluíram bem vizinhas de escolas nos maiores estratos de risco (primeiro por bairro, depois por raio);

5) Ver se o censo da educação contém a informação de quem era o diretor de cada escola em cada ano.



# Carrega os dados

In [193]:
import os
os.getcwd()

'/home/almeida2808/OneDrive/SeN/Ragazzo/Educação'

In [194]:
ideb_rio = pd.read_csv("./pr-educacao/data/treated/ideb_rio.csv")
proto_kepler = pd.read_csv("./pr-educacao/data/output/ideb_merged_kepler.csv")

In [195]:
proto_kepler.head()

,Unnamed: 0,ano,cod_inep,geometry,ideb,nome_abrev,nome_escola,lon,lat
0,0,2005-01-01 12:00:00,33140111,POINT (-43.63601828509798 -22.98632664816865),3.6,EM EMMA D'AVILA DE CAMILLIS,1026018 ESCOLA MUNICIPAL EMMA DAVILA DE CAMILLIS,-43.636018,-22.986327
1,1,2005-01-01 12:00:00,33140120,POINT (-43.63061141412631 -22.92095886212502),4.0,EM MANOEL PORTO FILHO,1019060 ESCOLA MUNICIPAL MANOEL PORTO FILHO,-43.630611,-22.920959
2,2,2005-01-01 12:00:00,33140154,POINT (-43.65817006396811 -22.93248534941013),4.0,EM PROF° JORGE GONÇALVES FARINHA,1019059 ESCOLA MUNICIPAL PROFESSOR JORGE GONCA...,-43.658170,-22.932485
3,3,2005-01-01 12:00:00,33146136,POINT (-43.62796962386766 -23.00163603612894),5.1,EM PROFª ELISA JOAQUINA DALTRO PEIXOTO,1026020 ESCOLA MUNICIPAL PROFESSORA ELISA JOAQ...,-43.627970,-23.001636
4,6,2005-01-01 12:00:00,33074763,POINT (-43.34848701721742 -22.84106860685418),4.6,EM VELINDA MAURÍCIO DA FONSECA,0515009 ESCOLA MUNICIPAL VELINDA MAURICIO DA F...,-43.348487,-22.841069


In [196]:
ideb_rio.shape

(867, 92)

In [197]:
ideb_rio.head()

,Unnamed: 0,sigla_uf,cod_municipio,nome_municipio,cod_inep,nome_escola,rede,taxa_aprovacao_2005_todos,taxa_aprovacao_2005_1,taxa_aprovacao_2005_2,...,ideb_2015,ideb_2017,projecoes_2007,projecoes_2009,projecoes_2011,projecoes_2013,projecoes_2015,projecoes_2017,projecoes_2019,projecoes_2021
0,38095,RJ,3304557.0,Rio de Janeiro,33062358.0,0101501 CIEP HENFIL,Municipal,100,100,100,...,4.1,4.7,4.4,4.7,5.1,5.4,5.6,5.9,6.1,6.4
1,38096,RJ,3304557.0,Rio de Janeiro,33062420.0,0101004 ESCOLA MUNICIPAL BENJAMIN CONSTANT,Municipal,76.4,100,100,...,4,5.1,4,4.3,4.7,5,5.3,5.5,5.8,6.1
2,38097,RJ,3304557.0,Rio de Janeiro,33062439.0,0101003 ESCOLA MUNICIPAL DARCY VARGAS,Municipal,99.2,100,100,...,5.6,5.2,4.3,4.7,5.1,5.3,5.6,5.9,6.1,6.4
3,38098,RJ,3304557.0,Rio de Janeiro,33062447.0,0101006 ESCOLA MUNICIPAL FRANCISCO BENJAMIM GA...,Municipal,77.3,81.7,91.2,...,5.4,5.6,3.5,3.8,4.2,4.5,4.8,5.1,5.4,5.7
4,38099,RJ,3304557.0,Rio de Janeiro,33062455.0,0101005 ESCOLA MUNICIPAL GENERAL MITRE,Municipal,87.9,95.2,98.7,...,5.8,5.9,4.5,4.8,5.2,5.5,5.7,6,6.2,6.5


# Identificar escolas que saltaram de IDEBs baixo/baixíssimo (< 5/< 4) e foram para IDEB alto (> 6) no período.

In [198]:
for column in ideb_rio.columns:
    if column[0:5] == "ideb_":
        ideb_rio[column] = ideb_rio[column].apply(lambda x: float(x) if x != "-" else np.nan)

In [199]:
ideb_rio["delta"] = ideb_rio["ideb_2017"] - ideb_rio["ideb_2005"]

In [200]:
ideb_rio["delta"].describe()

count    537.000000
mean       1.467784
std        0.647084
min       -0.300000
25%        1.000000
50%        1.400000
75%        1.800000
max        4.300000
Name: delta, dtype: float64

In [201]:
ideb_rio[['nome_escola', 'ideb_2005', 'ideb_2007', 'ideb_2009', 'ideb_2011', 'ideb_2013', 'ideb_2015', 'ideb_2017', 'delta']].sort_values("delta", ascending = False).head(15)

,nome_escola,ideb_2005,ideb_2007,ideb_2009,ideb_2011,ideb_2013,ideb_2015,ideb_2017,delta
770,0227501 CIEP DOUTOR BENTO RUBIAO,2.1,4.0,NaN,NaN,NaN,NaN,6.4,4.3
63,0206011 ESCOLA MUNICIPAL SHAKESPEARE,2.8,5.4,6.0,5.8,5.6,5.2,6.4,3.6
373,0734501 CIEP LUIZ CARLOS PRESTES,2.5,2.8,4.1,NaN,NaN,NaN,6.1,3.6
427,0716052 ESCOLA MUNICIPAL ROSA DO POVO,3.0,4.1,4.8,4.9,5.2,6.6,6.4,3.4
799,1026202 CIEP HILDEBRANDO DE ARAUJO GOES,2.1,3.9,4.8,5.2,5.1,5.1,5.5,3.4
500,0833019 ESCOLA MUNICIPAL LIMA BARRETO,3.2,3.0,3.7,5.7,5.4,5.9,6.5,3.3
28,0204003 ESCOLA MUNICIPAL ALBERTO BARTH,3.8,5.5,5.9,7.4,7.1,7.0,7.1,3.3
205,0411015 ESCOLA MUNICIPAL SUICA,3.6,4.8,5.6,5.3,5.5,6.8,6.7,3.1
48,0206501 CIEP PRESIDENTE JOAO GOULART,2.7,3.9,3.1,4.5,4.8,5.1,5.8,3.1
570,0918052 ESCOLA MUNICIPAL CASEMIRO DE ABREU,2.5,4.5,5.0,5.3,6.0,4.9,5.6,3.1


In [202]:
pessimo_pra_bom = ideb_rio[(ideb_rio['ideb_2005'] < 4) & (ideb_rio['ideb_2017'] > 6)].copy()
pessimo_pra_bom.shape

(24, 93)

In [203]:
pessimo_pra_bom.head()

,Unnamed: 0,sigla_uf,cod_municipio,nome_municipio,cod_inep,nome_escola,rede,taxa_aprovacao_2005_todos,taxa_aprovacao_2005_1,taxa_aprovacao_2005_2,...,ideb_2017,projecoes_2007,projecoes_2009,projecoes_2011,projecoes_2013,projecoes_2015,projecoes_2017,projecoes_2019,projecoes_2021,delta
23,38120,RJ,3304557.0,Rio de Janeiro,33063184.0,0103004 ESCOLA MUNICIPAL PEREIRA PASSOS,Municipal,80.7,100,98.3,...,6.3,3.9,4.3,4.7,5,5.2,5.5,5.8,6.1,2.4
25,38123,RJ,3304557.0,Rio de Janeiro,33063265.0,0204502 CIEP PRESIDENTE AGOSTINHO NETO,Municipal,90.6,93.1,94.4,...,6.5,3.6,3.9,4.3,4.6,4.9,5.2,5.5,5.8,3.0
28,38127,RJ,3304557.0,Rio de Janeiro,33064067.0,0204003 ESCOLA MUNICIPAL ALBERTO BARTH,Municipal,62.9,96.3,66.7,...,7.1,3.8,4.2,4.6,4.9,5.1,5.4,5.7,6,3.3
45,38145,RJ,3304557.0,Rio de Janeiro,33065004.0,0205007 ESCOLA MUNICIPAL PENEDO,Municipal,74.3,91.8,85.7,...,6.5,3.6,3.9,4.4,4.6,4.9,5.2,5.5,5.8,3.0
49,38149,RJ,3304557.0,Rio de Janeiro,33065225.0,0206502 CIEP NACAO RUBRO NEGRA,Municipal,69,78.2,87.6,...,6.1,3.3,3.7,4.1,4.4,4.6,4.9,5.2,5.5,2.8


In [204]:
ruim_pra_bom = ideb_rio[(ideb_rio['ideb_2005'] < 5) & (ideb_rio['ideb_2017'] > 6)].copy()
ruim_pra_bom.shape

(124, 93)

In [205]:
melhora_com_final_bom = ideb_rio[(ideb_rio['delta'] >= 2) & (ideb_rio['ideb_2017'] >= 6)]
melhora_com_final_bom.shape

(72, 93)

In [206]:
pessimo_pra_bom['cod_inep'].isin(ideb_rio.sort_values("delta", ascending = False).head(15)['cod_inep']).value_counts()

False    14
True     10
Name: cod_inep, dtype: int64

In [207]:
#cria listas com os cod_inep de cada um dos critérios pra poder incluir na visualização
pessimo_pra_bom_list = [int(x) for x in list(pessimo_pra_bom["cod_inep"])]
ruim_pra_bom_list = [int(x) for x in list(ruim_pra_bom["cod_inep"])]
melhora_com_final_bom_list = [int(x) for x in list(melhora_com_final_bom["cod_inep"])]

inicial_baixo_list = [int(x) for x in list(ideb_rio[ideb_rio["ideb_2005"] < 5]["cod_inep"])]
inicial_baixissimo_list = [int(x) for x in list(ideb_rio[ideb_rio["ideb_2005"] < 4]["cod_inep"])]

# Ver a visualização de dados olhando apenas para as escolas com IDEB inicial baixo (< 5) ou baixíssimo (< 4);

In [208]:
#cria flags pra usar de filtro com as categorias
proto_kepler["pessimo_pra_bom_bin"] = proto_kepler["cod_inep"].isin(pessimo_pra_bom_list)
proto_kepler["pessimo_pra_bom_bin"] = proto_kepler["pessimo_pra_bom_bin"].astype("int")

proto_kepler["ruim_pra_bom_bin"] = proto_kepler["cod_inep"].isin(ruim_pra_bom_list)
proto_kepler["ruim_pra_bom_bin"] = proto_kepler["ruim_pra_bom_bin"].astype("int")

proto_kepler["melhora_com_final_bom_bin"] = proto_kepler["cod_inep"].isin(melhora_com_final_bom_list)
proto_kepler["melhora_com_final_bom_bin"] = proto_kepler["melhora_com_final_bom_bin"].astype("int")

proto_kepler["inicial_baixo_bin"] = proto_kepler["cod_inep"].isin(inicial_baixo_list)
proto_kepler["inicial_baixo_bin"] = proto_kepler["inicial_baixo_bin"].astype("int")

proto_kepler["inicial_baixissimo_bin"] = proto_kepler["cod_inep"].isin(inicial_baixissimo_list)
proto_kepler["inicial_baixissimo_bin"] = proto_kepler["inicial_baixissimo_bin"].astype("int")

# Separar os dados em 4 extratos de IDEB para podermos ter uma noção de qual é o nível de atenção que precisa ser dispensado a cada escola;

In [209]:
#faz um quartile split
ideb_rio["nivel_atencao"] = pd.qcut(ideb_rio["ideb_2017"], 4, labels = [4, 3, 2, 1])

In [210]:
ideb_rio[["nome_escola", "nivel_atencao", "ideb_2017"]].head(15)

,nome_escola,nivel_atencao,ideb_2017
0,0101501 CIEP HENFIL,4,4.7
1,0101004 ESCOLA MUNICIPAL BENJAMIN CONSTANT,4,5.1
2,0101003 ESCOLA MUNICIPAL DARCY VARGAS,4,5.2
3,0101006 ESCOLA MUNICIPAL FRANCISCO BENJAMIM GA...,3,5.6
4,0101005 ESCOLA MUNICIPAL GENERAL MITRE,2,5.9
5,0101008 ESCOLA MUNICIPAL MARECHAL ESPERIDIAO R...,2,6.1
6,0101007 ESCOLA MUNICIPAL MARECHAL MASCARENHAS ...,NaN,NaN
7,0101804 EDI PROFESSORA LAURA SYLVIA MENDES PER...,NaN,NaN
8,0101009 ESCOLA MUNICIPAL PROFESSOR WALTER CARL...,NaN,NaN
9,0101001 ESCOLA MUNICIPAL VICENTE LICINIO CARDOSO,NaN,NaN


In [211]:
ideb_rio.groupby("nivel_atencao")["ideb_2017"].agg(["min", "mean", "max"])

,min,mean,max
nivel_atencao,,,
4,3.9,4.998742,5.3
3,5.4,5.609474,5.8
2,5.9,6.047945,6.2
1,6.3,6.622628,8.0


# Ver escolas que evoluíram bem vizinhas de escolas nos maiores estratos de risco

In [212]:
ideb_rio["cod_inep"] = ideb_rio["cod_inep"].apply(lambda x: int(x))

In [213]:
proto_kepler = pd.merge(proto_kepler, ideb_rio[['cod_inep', 'nivel_atencao']], how = "left", on = "cod_inep")

In [214]:
proto_kepler.to_csv("kepler_with_filters.csv", index = False)

Dada a visualização, o desafio agora é pegar, dada cada escola no pior quartil de IDEB (IDEB < 5.3), qual é a escola mais próxima das que foram de péssimo pra boa e de ruim pra péssimo?

In [215]:
proto_kepler.tail(3)

,Unnamed: 0,ano,cod_inep,geometry,ideb,nome_abrev,nome_escola,lon,lat,pessimo_pra_bom_bin,ruim_pra_bom_bin,melhora_com_final_bom_bin,inicial_baixo_bin,inicial_baixissimo_bin,nivel_atencao
4727,5716,2017-01-01 12:00:00,33082693,POINT (-43.67057996087902 -22.97906367786099),5.7,EM BERTHA LUTZ,1026010 ESCOLA MUNICIPAL BERTHA LUTZ,-43.670580,-22.979064,0,0,0,1,1,3
4728,5717,2017-01-01 12:00:00,33080810,POINT (-43.45065218838074 -22.86876566380888),5.1,EM MOACYR PADILHA,0817050 ESCOLA MUNICIPAL MOACYR PADILHA,-43.450652,-22.868766,0,0,0,1,0,4
4729,5718,2017-01-01 12:00:00,33140081,POINT (-43.60305310285084 -22.94811973568581),6.0,EM GIUSEPPE MELCHIORI - PE. JOSÉ,1026017 ESCOLA MUNICIPAL GIUSEPPE MELCHIORI - ...,-43.603053,-22.948120,0,0,0,1,0,2


In [216]:
geo_kepler = gpd.GeoDataFrame(proto_kepler, geometry=proto_kepler["geometry"].map(shapely.wkt.loads),
                             crs = {"init" : "epsg:4326"})

In [218]:
escolas_atencao = {}

for row, nivel in geo_kepler["nivel_atencao"].iteritems():
    escolas_referencia = geo_kepler.query("ruim_pra_bom_bin == 1 | melhora_com_final_bom_bin == 1 | pessimo_pra_bom_bin == 1").copy()
    escolas_referencia = escolas_referencia.query("ano == '2017-01-01 12:00:00'")
    escolas_referencia.to_crs(epsg=3395, inplace = True)
    if nivel == 4:
        escola_risco = geo_kepler["nome_escola"][row]
        escola_risco_df = geo_kepler[geo_kepler["nome_escola"] == escola_risco].copy()
        escola_risco_df.to_crs(epsg=3395, inplace = True)
        coord_escola_risco = escola_risco_df["geometry"][row]
        escolas_referencia["distancia"] = escolas_referencia["geometry"].apply(lambda x: x.distance(coord_escola_risco))
        escolas_referencia.sort_values("distancia", inplace = True)
        escolas_atencao[escola_risco] = escolas_referencia[[
            "nome_escola", "cod_inep", "ideb",
             "pessimo_pra_bom_bin", "ruim_pra_bom_bin",
            "melhora_com_final_bom_bin", "distancia"
        ]]

/home/almeida2808/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  frame[geo_column_name] = level
/home/almeida2808/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  frame[geo_column_name] = level
/home/almeida2808/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

/home/almeida2808/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  frame[geo_column_name] = level
/home/almeida2808/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  frame[geo_column_name] = level
/home/almeida2808/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

/home/almeida2808/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  frame[geo_column_name] = level
/home/almeida2808/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  frame[geo_column_name] = level
/home/almeida2808/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

/home/almeida2808/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  frame[geo_column_name] = level
/home/almeida2808/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  frame[geo_column_name] = level
/home/almeida2808/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

/home/almeida2808/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  frame[geo_column_name] = level
/home/almeida2808/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  frame[geo_column_name] = level
/home/almeida2808/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

...

/home/almeida2808/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  frame[geo_column_name] = level
/home/almeida2808/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  frame[geo_column_name] = level
/home/almeida2808/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

/home/almeida2808/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  frame[geo_column_name] = level
/home/almeida2808/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  frame[geo_column_name] = level
/home/almeida2808/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

/home/almeida2808/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  frame[geo_column_name] = level
/home/almeida2808/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  frame[geo_column_name] = level
/home/almeida2808/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

/home/almeida2808/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  frame[geo_column_name] = level
/home/almeida2808/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  frame[geo_column_name] = level
/home/almeida2808/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

/home/almeida2808/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  frame[geo_column_name] = level
/home/almeida2808/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  frame[geo_column_name] = level
/home/almeida2808/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

In [219]:
escolas_atencao.keys()

dict_keys(['1019064 ESCOLA MUNICIPAL ALVARO VALLE', '0622020 ESCOLA MUNICIPAL MAURICE MAETERLINCK', '0625035 ESCOLA MUNICIPAL PREFEITO MARCOS TAMOYO', '0430002 ESCOLA MUNICIPAL TEOTONIO VILELA', '0430501 CIEP PRESIDENTE SAMORA MACHEL', '0430502 CIEP ELIS REGINA', '0430206 CIEP HELIO SMIDT', '0430201 CIEP MINISTRO GUSTAVO CAPANEMA', '0410023 ESCOLA MUNICIPAL LUIZ CESAR SAYAO GARCEZ', '1026009 ESCOLA MUNICIPAL PADRE JOSE MAURICIO', '1120021 ESCOLA MUNICIPAL GURGEL DO AMARAL', '1120502 CIEP JOAO MANGABEIRA', '1120503 CIEP DOUTOR JOAO RAMOS DE SOUZA', '0209021 ESCOLA MUNICIPAL PRESIDENTE JOAO GOULART', '0312503 CIEP CORONEL SARMENTO', '0313044 ESCOLA MUNICIPAL JOAO KOPKE', '0328501 CIEP VINICIUS DE MORAES', '0410502 CIEP MAESTRO FRANCISCO MIGNONE', '0431017 ESCOLA MUNICIPAL CARDEAL CAMARA', '0622004 ESCOLA MUNICIPAL PARAIBA', '0625006 ESCOLA MUNICIPAL ERICO VERISSIMO', '0625204 CIEP RUBENS GOMES', '0431501 CIEP MESTRE CARTOLA AGENOR DE OLIVEIRA', '0411502 CIEP DEPUTADO CARLOS BRANDAO MONTE

In [220]:
escolas_atencao["1019064 ESCOLA MUNICIPAL ALVARO VALLE"].head()

,nome_escola,cod_inep,ideb,pessimo_pra_bom_bin,ruim_pra_bom_bin,melhora_com_final_bom_bin,distancia
4433,1019020 ESCOLA MUNICIPAL PROFESSORA MARIA SANT...,33085013,6.9,0,1,1,2048.957049
4435,1019010 ESCOLA MUNICIPAL JOSE DE MELLO,33084785,6.3,0,1,0,2266.673280
4217,1019205 CIEP ROBERTO MORENA,33084149,6.1,1,1,1,3000.396936
4344,1019033 ESCOLA MUNICIPAL ESPANHA,33084653,6.2,0,1,1,3756.842875
4346,1019032 ESCOLA MUNICIPAL MIGUEL CALMON,33084866,6.1,0,1,0,4127.942995


In [221]:
#imprime os dfs de cada uma das escolas
for school in escolas_atencao:
    escolas_atencao[school].to_csv("./sponsor_partners/" + school + ".csv", index = False)